# BRAG Demo

This notebook will re-create all figures and statistical analyses presented in Hann-Soden et al (2018).

In [17]:
# Set parameters
step = 4000
window_size = 20000

genome_file = 'genomes/Neurospora-crassa_OR74A_v12_fixed.fasta'
annotation_file = 'genomes/annotations/Neurospora_crassa_OR74A_v12_fixed.gtf'
centromere_file = 'Neurospora-crassa_v12_fixed_centromeres.txt'
segment_files = 'alignments/Neurospora.*.os.txt'
seqs_files = 'alignments/Neurospora.*.seqs'
tree_file = 'Neurospora.nwk'
reference_name = 'Nc'
outgroup_name = 'Sm'
output_prefix = 'Ncrassa'
core_gene_file = 'takao_core_genes.tsv'
core_gene_key = 'takao_core_genes_key.txt'

In [8]:
# Set up the environment
import sys, os, glob
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import gridspec
from scipy import stats
import statsmodels.stats.multitest as smm

from BRAG_parsers import scaffold_table
from fasta_tools import fasta_to_dict
from gff_tools import gff_table
from plots import regression_plot, pretty_bar
from plotting_tools import direct_labels

BRAGdir = os.getcwd()

sys.path.append(BRAGdir)
sys.path.append(BRAGdir+'/hannsoden-bioinformatics')

os.chdir('sample_data')

In [9]:
# Download the genomes used in the study
import urllib.request, tarfile

genome_source = 'https://osf.io/vmkc8/?action=download'
genome_target = 'genomes.tar.gz'
response = urllib.request.urlretrieve(genome_source, genome_target)

tar = tarfile.open(genome_target, 'r:gz')
tar.extractall()
tar.close()

## Generate the extra tracks used

In [10]:
def windowize(genome, step=4000, window_size=20000):
    # Produce start and end coordinates of sliding windows
    start = 0
    end = start + window_size
    genlen = len(genome)
    while end < genlen:
        yield (start, end)
        start += step
        end += step

def flatten(df):
    # Merge overlapping regions into a single presence/absence layer.
    # Assumes df is sorted.
    regions = {'start':[], 'end':[]}
    region_start = 0
    region_end = 0
    for index, row in df.iterrows():
        if row.abs_start > region_end:
            regions['start'].append(region_start)
            regions['end'].append(region_end)
            region_start = row.abs_start
        region_end = row.abs_end
    regions['start'] = regions['start'][1:]
    regions['end'] = regions['end'][1:]
    return pd.DataFrame(regions)

def GC(seq):
    gc = seq.count('G') + seq.count('C')
    gc = float(gc) / len(seq)
    return gc

def CRI(seq):
    # composite rip index = (TA/AT)- (CA+TG)/(AC+GT)
    TA = float(seq.count('TA'))+1
    AT = float(seq.count('AT'))+1
    CA = float(seq.count('CA'))+1
    TG = float(seq.count('TG'))+1
    AC = float(seq.count('AC'))+1
    GT = float(seq.count('GT'))+1
    return (TA/AT) - (CA+TG)/(AC+GT)

def gene_density(annotation, window):
    start, end = window
    window_size = float(end - start)
    overlapping = annotation.loc[(annotation.start < end) & (annotation.end > start)]
    start = np.full( len(overlapping), start, dtype=int)
    starts = np.maximum(start, overlapping.start)
    end = np.full( len(overlapping), end, dtype=int)
    ends = np.minimum(end, overlapping.end)
    overlap = np.sum(ends - starts)
    return overlap / window_size

# Read in the reference genome and concatenate it into a single string.
fasta = open(genome_file, 'r').read()
scaffolds = fasta[1:].split('>')
scaffolds = [s.split('\n', 1) for s in scaffolds]
scaffolds = [s[1].replace('\n', '') for s in scaffolds]
genome = ''.join(scaffolds)

# Read in the annotation and calculate coordinates for features based on
# the concatenated genome sequence, rather than for each scaffold.
annotation = gff_table(annotation_file)
scaffolds = scaffold_table(genome_file)
abs_positions = {row['name']: row.abs_pos for i, row in scaffolds.iterrows()}
scaf_adjustment = annotation.seqname.replace(abs_positions)
annotation['abs_start'] = annotation.start + scaf_adjustment
annotation['abs_end'] = annotation.end + scaf_adjustment
annotation.sort_values('abs_start', inplace=True)

# Find protein coding and expressed regions of the genome.
# Flattening avoids double counting regions that are in multiple transcripts.
protein_coding = annotation.loc[annotation.feature == 'CDS']
protein_coding = flatten(protein_coding)
expressed = annotation.loc[annotation.feature == 'exon']
expressed = flatten(expressed)

# Calculate different statistics for each window, put into dataframe.
columns = ['start', 'end', 'GC', 'CRI', 'cds_density', 'exon_density']
tracks = {column:[] for column in columns}
for window in windowize(genome, window_size=window_size, step=step):
    seg = genome[slice(*window)]
    seg = seg.upper()
    tracks['start'].append( window[0] )
    tracks['end'].append( window[1] )
    tracks['GC'].append( GC(seg) )
    tracks['CRI'].append( CRI(seg) )
    tracks['cds_density'].append( gene_density(protein_coding, window) )
    tracks['exon_density'].append( gene_density(expressed, window) )

tracks = pd.DataFrame(tracks)
print(tracks)
tracks.to_csv('extra_tracks.tsv', sep='\t', columns = columns, index=False)

# CRI and GC are colinear, basically the same thing, so both shouldn't be used for
# break rate regression. Exon density and CDS density are also highly related.
# Exon density is also heavily affected by RIP, so I want to include exon density
# only at un-ripped sites.
# Positive scores for CRI indicate RIP.
tracks['unripped_exon_density'] = tracks.exon_density
tracks.loc[tracks.CRI > 0, 'unripped_exon_density'] = np.NaN

# Write dataframe to file.
tracks.to_csv('extra_tracks_unique.tsv', sep='\t',
              columns = ['start', 'end', 'CRI', 'unripped_exon_density'],
              index=False)


ModuleNotFoundError: No module named 'fasta_tools'

## Run BRAG

In [23]:
# This would be the command used to run BRAG
command = (BRAGdir+'/BRAG -t '+tree_file+' -r '+reference_name+
           ' -x '+outgroup_name+' -o '+output_prefix+
           ' -g '+segment_files+' -q '+seqs_files+
           ' -C '+centromere_file+' -T extra_tracks.tsv '+
           ' -W '+str(window_size)+' -S '+str(step))
print(command)

/home/christopher/BRAG/BRAG -t Neurospora.nwk -r Nc -x Sm -o Ncrassa -g alignments/Neurospora.*.os.txt -q alignments/Neurospora.*.seqs -C Neurospora-crassa_v12_fixed_centromeres.txt -T extra_tracks.tsv  -W 20000 -S 4000


In [ ]:
# Run BRAG from Python
from BRAG_main import main as BRAG

seqs = glob.glob(seqs_files)
segs = glob.glob(segment_files)

BRAG(tree_file, reference_name, outgroup_name, output_prefix,
     segs, seqs, step=step, window_size=window_size,
     centromeres=centromere_file, tracks='extra_tracks.tsv')

In [ ]:
# Run again with unique tracks
BRAG(tree_file, reference_name, outgroup_name, output_prefix+'_unique',
     segs, seqs, step=step, window_size=window_size,
     centromeres=centromere_file, tracks='extra_tracks_unique.tsv')

## Analyze results & make figures

In [ ]:
def telomere_correlations(dataset, chromosomes, datalabel):
    dataset['midpoint'] = (dataset.start + dataset.end) / 2.
    tests = []
    for chr_num, coordinates in enumerate(chromosomes):
        label = 'Chr_{}_'.format(chr_num+1)
        lstart, lend, rstart, rend = coordinates # coordinates of left and right arms of the chromosomes

        larm = dataset.loc[(dataset.start >= lstart) & (dataset.end <= lend)]
        rarm = dataset.loc[(dataset.start >= rstart) & (dataset.end <= rend)]

        # left arm works from left to right
        ltest = regression_plot(larm.midpoint - lstart, larm[datalabel], label=label+'left')
        ltest.regress(slope = 'negative')
        tests.append(ltest)
        # right arm works from right to left
        rtest = regression_plot(rend - rarm.midpoint, rarm[datalabel], label=label+'right')
        rtest.regress(slope = 'negative')
        tests.append(rtest)
    return tests

def read_centromeres(cen_file, chromosomes):
    fh = open(cen_file, 'r')
    header = fh.readline()
    idxs, starts, stops = list(zip(*[list(map(int, line.split('#')[0].strip().split())) for line in fh]))
    centromeres = [[0, max(stops), 0, 0] for x in range(max(idxs)+1)]
    for idx, start, stop in zip(idxs, starts, stops):
        centromeres[idx][0] = chromosomes.iloc[idx].abs_pos
        centromeres[idx][1] = chromosomes.iloc[idx].abs_pos + min(start, centromeres[idx][1])
        centromeres[idx][2] = chromosomes.iloc[idx].abs_pos + max(stop, centromeres[idx][2])
        centromeres[idx][3] = chromosomes.iloc[idx+1].abs_pos -1
    return centromeres

certain = pd.read_csv(output_prefix+'_certain_rate_windows.txt', sep='\t', header=0)
uncertain = pd.read_csv(output_prefix+'_uncertain_rate_windows.txt', sep='\t', header=0)
# get the scaffold lengths
scaffolds = scaffold_table(genome_file)
# calculate the coordinates of left and right arms of the chromosomes
chromosomes = read_centromeres(centromere_file, scaffolds)

# look at gene content in high and low regions
abs_positions = {row['name']: row.abs_pos for i, row in scaffolds.iterrows()}
annotation = gff_table(annotation_file)
annotation = annotation.loc[annotation.feature == 'CDS'] # I only care about the CDS, not the start codons & exons etc
scaf_adjustment = annotation.seqname.replace(abs_positions)
annotation['abs_start'] = annotation.start + scaf_adjustment
annotation['abs_end'] = annotation.end + scaf_adjustment

def top_regions(data, annotation, top_size, outfile, ascending=False):
    data = data.sort_values('E', ascending=ascending)
    i = 0
    total_length = 0
    while total_length < top_size:
        i += 1
        top_windows = data.head(i)
        top_regions, total_length = collapse_regions(top_windows)

    fh = open(outfile, 'w')
    geneIDs = set()
    for start, end, length in top_regions:
        features = annotation.loc[(annotation.abs_start <= end) & (annotation.abs_end >= start)]
        genes = set(features.attributes)
        geneIDs |= set([gene.split()[1][1:-2] for gene in genes])
        fh.write( 'Region {}-{} ({} bp): {} genes\n'.format(start, end, length, len(genes)) )
        for gene in genes:
            fh.write( '{}\n'.format(gene) )
        fh.write('\n')
    fh.close()

    return geneIDs, total_length

def collapse_regions(data):
    data = data.sort_values('start')
    regions = []
    total_length = 0
    region_start = 0
    region_end = 0
    for index, row in data.iterrows():
        if row.start > region_end:
            region_length = region_end - region_start
            total_length += region_length
            regions.append( (region_start, region_end, region_length) )
            region_start = row.start
        region_end = row.end
    regions = regions[1:]
    return regions, total_length

most_size  = 2000000
# Conserved regions
low_genes, conserved_length = top_regions(certain, annotation, most_size, 'conserved_regions.txt', ascending=True)
print('{} genes found in {} bp of conserved regions'.format(len(low_genes), conserved_length))
high_genes, fragile_length = top_regions(certain, annotation, most_size, 'fragile_regions.txt', ascending=False)
print('{} genes found in {} bp of fragile regions'.format(len(high_genes), fragile_length))

def parse_core_genes(gene_table, key_file):
    gene_table = pd.read_csv(gene_table, delimiter='\t', header=0)
    keyfh = open(key_file, 'r')
    ranks = {}
    better_labels = {}
    for line in keyfh:
        rank, code, description = line.strip().split('\t')
        ranks[code] = int(rank)
        better_labels[code] = description
    gene_table['coreness'] = gene_table.ID30
    gene_table.coreness.replace(ranks, inplace=True)
    gene_table.ID30.replace(better_labels, inplace=True)
    gene_table.sort_values('coreness', inplace=True)

    return gene_table

def core_enrichment(genes, core_genes):
    core_genes_present = core_genes.loc[core_genes.Broad7_geneID.isin(genes)]

    category_counts = {}
    label_order = []
    for i, row in core_genes_present.iterrows():
        try:
            category_counts[row.ID30] += 1
        except KeyError:
            category_counts[row.ID30] = 1
            label_order.append(row.ID30)
    label_order = [l for l in label_order if l != 'Others']

    return category_counts, label_order

# Compare core gene content of high and low genes
print()
core_genes = parse_core_genes(core_gene_file, core_gene_key)
lowcounts, label_order = core_enrichment(low_genes, core_genes)
highcounts, label_order = core_enrichment(high_genes, core_genes)
label_order = label_order[::-1]
diffcounts = [lowcounts[label]-highcounts[label] for label in label_order]


fig = plt.figure(figsize = (6, 6))
ax = fig.add_subplot(111)
bars = pretty_bar(ax, diffcounts, label_order, horizontal=True)
direct_labels(ax, list(range(len(diffcounts))), diffcounts, horizontal=True, num_format='0:+')
direct_labels(ax, list(range(len(diffcounts))),
              [((n<0)*2)-1 for n in diffcounts],
              altlabels=[highcounts[l] for l in label_order],
              horizontal=True)
ax.set_xlim(-105, 105)
ax.set_xlabel('Conserved Genes - Fragile Genes')
fig.savefig('core_gene_content')

core_genes_in_category = {}
for i, row in core_genes.iterrows():
    try:
        core_genes_in_category[row.ID30] += 1
    except KeyError:
        core_genes_in_category[row.ID30] = 1
print('{} genes have a published phylogenetic distribution'.format(sum(core_genes_in_category.values())))
for category, number in list(core_genes_in_category.items()):
    print('{}\t{}'.format(number, category))

print('{} / {} genes in conserved regions have published phylogenetic distribution'.format(sum(lowcounts.values()), len(low_genes)))
print('{} / {} genes in fragile regions have published phylogenetic distribution'.format(sum(highcounts.values()), len(high_genes)))
print()

distributions = [[lowcounts[label] for label in label_order],
                 [highcounts[label] for label in label_order]]
chi2, pval, dof, expected = stats.chi2_contingency(distributions)
print('2-sample Chi-squared test if the distribution of phylogenetic conservation of')
print('genes in fragile (most rapidly breaking) and conserved regions are equal:')
print('chi2', chi2)
print('pval', pval)
print('dof', dof)
print()


# is break rate correlated with distance to telomere?

tests = telomere_correlations(certain, chromosomes, 'E')
#tests.extend( telomere_correlations(uncertain, chromosomes, 'E') )

# multiple testing correction for testing each chromosome arm independently
# use 'fdr_tsbh' for two stage fdr correction via Benjamini/Hochberg method
p_vals = [test.raw_slope_p for test in tests]
rejects, p_vals, bs, nonsense = smm.multipletests(p_vals, alpha=0.05, method='fdr_tsbh')


sig_tests = []
insig_tests = []
print('label\traw_p\tp\tslope\tintercept\tr_squared')
for test, pv, reject in zip(tests, p_vals, rejects):
    test.p_val = pv
    print('{}\t{:.2E}\t{:.2E}\t{:.2E}\t{:.2E}\t{:0.4f}'.format(test.label, test.raw_slope_p, pv, test.slope, test.intercept, test.r2))
    if reject:
        sig_tests.append(test)
    else:
        insig_tests.append(test)

print('sig tests', len(sig_tests))
print('insig tests', len(insig_tests))

def grid_plots(tests, outfile, logy=True):
    nplots = len(tests)
    columns = 2
    rows = int((nplots -1) / 2) + 1
    plotsize = (4, 4)
    figsize = (plotsize[0]*columns, plotsize[1]*rows)
    fig = plt.figure(figsize=figsize)
    gs = gridspec.GridSpec(rows, columns)
    axes = [fig.add_subplot(gs[x]) for x in range(nplots)]

    for ax, test in zip(axes, tests):
        test.draw(ax, logy = logy, fit_report_location = (0.05, 0.05))
    fig.savefig(outfile, dpi=350)
    plt.close(fig)

grid_plots(tests, 'brag_correlations')

# is CRI correlated with distance to telomere?

print('regression of CRI w/ telomere distance')

extra_tracks = pd.read_csv('Ncra_extra_tracks.tsv', sep='\t', header=0)

tests = telomere_correlations(extra_tracks, chromosomes, 'CRI')

# multiple testing correction for testing each chromosome arm independently
# use 'fdr_tsbh' for two stage fdr correction via Benjamini/Hochberg method
p_vals = [test.raw_slope_p for test in tests]
rejects, p_vals, bs, nonsense = smm.multipletests(p_vals, alpha=0.05, method='fdr_tsbh')


sig_tests = []
insig_tests = []
print('label\traw_p\tp\tslope\tintercept\tr_squared')
for test, pv, reject in zip(tests, p_vals, rejects):
    test.p_val = pv
    print('{}\t{:.2E}\t{:.2E}\t{:.2E}\t{:.2E}\t{:0.4f}'.format(test.label, test.raw_slope_p, pv, test.slope, test.intercept, test.r2))
    if reject:
        sig_tests.append(test)
    else:
        insig_tests.append(test)

print('sig tests', len(sig_tests))
print('insig tests', len(insig_tests))

grid_plots(tests, 'CRI_correlations', logy=False)